In [ ]:
import requests
import pandas as pd
from sqlalchemy import create_engine

# AviationStack API Key (Replace with your actual key)
API_KEY = "e12f68b0fbc64150dd9feca71d7f3101"

# PostgreSQL Database Connection (Modify as needed)
DB_USER = "postgres"
DB_PASSWORD = "password"
DB_HOST = "localhost"  # Change if hosted elsewhere
DB_PORT = "5432"  # Default PostgreSQL port
DB_NAME = "postgres"

# Create database engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# API Request URL for live flights from Scottsdale Executive Airport (SCF)
url = f"http://api.aviationstack.com/v1/flights?access_key=e12f68b0fbc64150dd9feca71d7f3101&dep_iata=SCF"

# Fetch data
response = requests.get(url)
data = response.json()

# Convert API response to DataFrame
flights = data.get("data", [])
df = pd.DataFrame(flights)

if df.empty:
    print("No flight data found.")
else:
    # Extract relevant fields
    df["flight_number"] = df["flight"].apply(lambda x: x.get("number") if isinstance(x, dict) else None)
    df["airline"] = df["airline"].apply(lambda x: x.get("name") if isinstance(x, dict) else None)
    df["departure_airport"] = df["departure"].apply(lambda x: x.get("airport") if isinstance(x, dict) else None)
    df["arrival_airport"] = df["arrival"].apply(lambda x: x.get("airport") if isinstance(x, dict) else None)
    df["departure_time"] = df["departure"].apply(lambda x: x.get("estimated") if isinstance(x, dict) else None)
    df["arrival_time"] = df["arrival"].apply(lambda x: x.get("estimated") if isinstance(x, dict) else None)
    df["status"] = df["flight_status"]

    # Keep only the relevant columns
    df_cleaned = df[["flight_number", "airline", "departure_airport", "arrival_airport", "departure_time", "arrival_time", "status"]]

    # Insert into database
    df_cleaned.to_sql("flights", engine, if_exists="append", index=False)

    print("Data successfully inserted into the database.")


In [ ]:
import requests
import pandas as pd
from sqlalchemy import create_engine

# AviationStack API Key (Replace with your actual key)
API_KEY = "e12f68b0fbc64150dd9feca71d7f3101"

# PostgreSQL Database Connection (Modify as needed)
DB_USER = "postgres"
DB_PASSWORD = "password"
DB_HOST = "localhost"  # Change if hosted elsewhere
DB_PORT = "5432"  # Default PostgreSQL port
DB_NAME = "postgres"

# Create database engine
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# API Request URL for live flights from Scottsdale Executive Airport (SCF)
url = f"http://api.aviationstack.com/v1/flights?access_key=e12f68b0fbc64150dd9feca71d7f3101&dep_iata=SCF"

# Fetch flight data
response = requests.get(url)
data = response.json()

# Extract flights data
flights = data.get("data", [])

if not flights:
    print("No flight data found.")
else:
    # Step 1: Loop through each flight and fetch airplane details
    airplanes_data = []
    for flight in flights:
        aircraft = flight.get("aircraft")
        if aircraft:
            registration_number = aircraft.get("registration")
            if registration_number:
                # Step 2: Query the airplanes API for aircraft details
                airplane_url = f"https://api.aviationstack.com/v1/airplanes?access_key=e12f68b0fbc64150dd9feca71d7f3101"
                airplane_response = requests.get(airplane_url)
                airplane_data = airplane_response.json()

                if airplane_data.get("data"):
                    airplane = airplane_data["data"][0]
                    airplanes_data.append(airplane)

    # Step 3: Convert the airplane data into a DataFrame
    if airplanes_data:
        df_airplanes = pd.DataFrame(airplanes_data)

        # Step 4: Insert data into PostgreSQL database
        df_airplanes.to_sql("airplanes", engine, if_exists="append", index=False)
        print("Airplane data successfully inserted into the database.")
    else:
        print("No airplane data found for the flights.")


In [ ]:
import requests
import psycopg2
from datetime import datetime

# API endpoint and your API key
API_KEY = 'e12f68b0fbc64150dd9feca71d7f3101'
BASE_URL = 'http://api.aviationstack.com/v1/airplanes'

# Database connection details
DB_HOST = 'localhost'
DB_NAME = 'postgres'
DB_USER = 'postgres'
DB_PASSWORD = 'password)'

# Function to fetch flight data from AviationStack
def fetch_flight_data():
    params = {
        'access_key': API_KEY,
    }
    
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        return response.json()['data']
    else:
        print(f"Error fetching data: {response.status_code}")
        return []

# Function to insert flight data into the PostgreSQL database
def insert_flight_data(data):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        cursor = connection.cursor()
        
        # SQL insert statement
        insert_query = """
            INSERT INTO airplanes (
                registration_number, iata_type, model_name, model_code,
                registration_date, line_number, plane_series, airline_iata_code,
                airline_icao_code, plane_owner, engines_count, engines_type, 
                plane_age, plane_status, plane_class
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
        """

        # Prepare data for insertion
        records = []
        for airplane in data:
            # Extract relevant data
            registration_number = airplane.get('registration_number', None)
            iata_type = airplane.get('iata_type', None)
            model_name = airplane.get('model_name', None)
            model_code = airplane.get('model_code', None)
            
            # Convert registration_date to datetime format if available
            registration_date = airplane.get('registration_date', None)
            if registration_date:
                registration_date = datetime.strptime(registration_date, "%Y-%m-%d").date()
            
            line_number = airplane.get('line_number', None)
            plane_series = airplane.get('plane_series', None)
            airline_iata_code = airplane.get('airline_iata_code', None)
            airline_icao_code = airplane.get('airline_icao_code', None)
            plane_owner = airplane.get('plane_owner', None)
            engines_count = airplane.get('engines_count', None)
            engines_type = airplane.get('engines_type', None)
            plane_age = airplane.get('plane_age', None)
            plane_status = airplane.get('plane_status', None)
            plane_class = airplane.get('plane_class', None)
            
            # Create record tuple
            record = (
                registration_number, iata_type, model_name, model_code, 
                registration_date, line_number, plane_series, airline_iata_code, 
                airline_icao_code, plane_owner, engines_count, engines_type, 
                plane_age, plane_status, plane_class
            )
            records.append(record)
        
        # Insert data into the database
        cursor.executemany(insert_query, records)
        connection.commit()
        print(f"{len(records)} records inserted successfully.")
    
    except Exception as error:
        print(f"Error inserting data: {error}")
    finally:
        if connection:
            cursor.close()
            connection.close()

# Main function to run the script
def main():
    # Fetch flight data from the API
    flight_data = fetch_flight_data()
    
    if flight_data:
        # Insert flight data into the PostgreSQL database
        insert_flight_data(flight_data)

if __name__ == "__main__":
    main()
